In [2]:
import numpy as np 
import pandas as pd
import cvxpy as cp

In [3]:
data=pd.read_excel("./附件.xlsx")

In [4]:
one_point_cbts = (2812-200)/3
one_point_epds = (1890-500)/2
one_point_hads = (12500-300)/5

In [5]:
score_cbts_238 = data[data["编号"]==238]["CBTS"].values[0]
score_epds_238 = data[data["编号"]==238]["EPDS"].values[0]
score_hads_238 = data[data["编号"]==238]["HADS"].values[0]


In [6]:
data[data["编号"]==238]

,编号,母亲年龄,婚姻状况,教育程度,妊娠时间（周数）,分娩方式,CBTS,EPDS,HADS,婴儿行为特征,...,整晚睡眠时间（时：分：秒）,睡醒次数,入睡方式,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
237,238,24,2,2,26.5,1,15,22,18,矛盾型,...,09:00:00,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data.describe()

,编号,母亲年龄,婚姻状况,教育程度,妊娠时间（周数）,分娩方式,CBTS,EPDS,HADS,婴儿性别,婴儿年龄（月）,睡醒次数,入睡方式,Unnamed: 15
count,410.000000,410.000000,410.000000,410.000000,410.000000,410.000000,410.000000,410.000000,410.000000,410.000000,410.000000,390.000000,390.000000,0.0
mean,205.500000,30.197561,2.002439,4.080488,39.114146,1.012195,5.943902,9.048780,7.839024,1.482927,1.958537,1.461538,3.025641,NaN
std,118.501055,4.362262,0.359970,1.001642,1.900777,0.109890,4.963365,6.757595,4.259715,0.500319,0.821911,1.612071,1.408516,NaN
min,1.000000,19.000000,1.000000,1.000000,26.500000,1.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,1.000000,NaN
25%,103.250000,27.000000,2.000000,3.000000,38.000000,1.000000,2.000000,4.000000,5.000000,1.000000,1.000000,0.000000,2.000000,NaN
50%,205.500000,30.000000,2.000000,4.000000,39.200000,1.000000,5.000000,7.000000,7.000000,1.000000,2.000000,1.000000,4.000000,NaN
75%,307.750000,33.000000,2.000000,5.000000,40.400000,1.000000,9.000000,13.000000,11.000000,2.000000,3.000000,2.000000,4.000000,NaN
max,410.000000,47.000000,6.000000,5.000000,43.000000,2.000000,21.000000,28.000000,20.000000,2.000000,3.000000,10.000000,5.000000,NaN


In [8]:
##其中我们可以看出这几个指标的平均值大约为6，9，8
##因为得分均为整数因此这个规划问题为整数规划问题
x = cp.Variable(3,integer = True)

In [13]:
cost = one_point_cbts*x[0] + one_point_epds * x[1] + one_point_hads *x[2] +1000
objective = cp.Minimize(cost)
##采用岭回归进行分类 其预测结果的函数为：
##婴儿行为特征=0.541＋0.109 × 婚姻状况-0.003 × HADS＋0.008 × EPDS-0.008 × CBTS-0.02 × 教育程度-0.3 × 分娩方式＋0.003 × 妊娠时间（周数）＋0.002 × 母亲年龄
f = 0.541+0.109*2-0.003*x[2]+0.008*x[1]-0.008*x[0]-0.02*2-0.3*1+0.003*26.5+0.002*24
cons = [
    x[0]>=0,
    x[1]>=0,
    x[2]>=0,
    score_cbts_238-x[0]>=0,
    score_cbts_238 -x[0]<=6,
    score_epds_238 -x[1]>=0,
    score_epds_238-x[1]<=9,
    score_hads_238 - x[2]>=0,
    score_hads_238 - x[2]<=8,
    f>=0.5,
    f<=1.5
    
    
]
prob=cp.Problem(objective,cons)
prob.solve(solver='CPLEX')
print("最优值为：",prob.value)
print("最优解为：",x.value)

最优值为： 42271.0
最优解为： [ 9. 13. 10.]


In [14]:
## 如果要变成安静形的话我们只需要更改约束条件
cons2 = [
    x[0]>=0,
    x[1]>=0,
    x[2]>=0,
    score_cbts_238-x[0]>=0,
    score_cbts_238 -x[0]<=6,
    score_epds_238 -x[1]>=0,
    score_epds_238-x[1]<=9,
    score_hads_238 - x[2]>=0,
    score_hads_238 - x[2]<=8,
    f>=0,
    f<=0.5
    
    
]
prob=cp.Problem(objective,cons2)
prob.solve(solver='CPLEX')
print("最优值为：",prob.value)
print("最优解为：",x.value)

最优值为： 49935.0
最优解为： [15. 13. 11.]
